In [1]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials



# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_list = drive.ListFile({'q': "'14_dERITvngnhPUZnXXuatSXyX7h-LslO' in parents"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))


normal_file_list = drive.ListFile({'q': "'1_uOwvtnnJtPuqzmLyljEKBpqXo9dnBzJ' in parents"}).GetList()
#for filen in normal_file_list:
#    print('title: %s, id: %s' % (filen['title'], filen['id']))


abnormal_file_list = drive.ListFile({'q': "'1wPMSqW8g1ntxU3vsXEFOb_TSa5MaLyzZ' in parents"}).GetList()
#for filea in abnormal_file_list:
  #print('title: %s, id: %s' % (filea['title'], filea['id']))


test_file_list = drive.ListFile({'q': "'1_GC8zEsBx1DiSCKqP-JX_Vn3MybI9NBM' in parents"}).GetList()
#for filet in test_file_list:
   # print('title: %s, id: %s' % (filet['title'], filet['id']))

#for f in abnormal_file_list:
#  f_ = drive.CreateFile({'id': f['id']})
#  f_.GetContentFile(f['title'])
   

  
# Import relavent libraries. Using Keras for the neural network

import os
import keras
import tensorflow as tf
import pandas as pd
from PIL import Image
import numpy as np
import zipfile
import io
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import SGD
from google.colab import files


#uploaded = files.upload()
#data=zipfile.ZipFile(io.BytesIO(uploaded['dataDir']), 'r')
#data.extractall()
#data.printdir()

#print (uploaded)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#Data is split into "normal" and "abnormal" for training, the neural network 
#needs examples of anomalous and normal data

#normalData = 'fireworks/medium/normal/'
abnormalData= 'dataDir/abnormal'
#abnormalData = 'fireworks/medium/abnormal/'

#this is the data to test if the classifier works, uses 
testData='dataDir/test/'
#testData = 'fireworks/medium/test/'


#arrays to parse images to
normalArray = []
abnormalArray = []
testArray = []

normal_Label=[]
abnormal_Label=[]

for f in abnormal_file_list:
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(f['title'])
  img = Image.open(f['title'])  
  img.load()
  data = np.array(img)
  gray=data[:,:,0]
  print(data.shape)
  abnormalArray.append(data)

#fx = drive.CreateFile({'id': '1SmPU9pnXmCjPOOrylkVNX7ivywDmO6Qj'})
#fx.GetContentFile(fx['title'])
#print(fx)
#df=pd.read_csv(fx['title'])
#print (df.values)
#normalArray = df.values


for f in file_list:
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(f['title'])
    img = Image.open(f['title'])  
    img.load()
    data = np.array(img)
    print(np.array(data).shape)
    normalArray.append(data)
  
for f in test_file_list:
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(f['title'])
  img = Image.open(f['title'])  
  img.load()
  data = np.array(img)
  gray=data[:,:,0]
  print(gray.shape)
  testArray.append(gray)  
  
#Creating a neural network, it is done layer by layer, the input_shape is the shape
#of the images i.e 540x960, each layer represents a computation with the input
#and is fed forward to the next layer. Flatten() is used to reduce the dimensions of the input data, 
#you would typically do this before the last layer which returns the output
#The "loss" represents the "price paid" for inaccuracy of predictions, there are various functions that
#can be used depending on the model/problem

def model_load():
    model = Sequential()
    model.add(Dense(units=128, input_shape=(540, 960)))
    model.add(Dense(4, input_dim=2, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

def conv_model_load():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(540, 960, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    print(model.summary())
    return model

#Simple image to array conversion for normal/abnormal/test
def normal_conv_Image(fileName):
    img = Image.open(normalData+fileName)
    img.load()
    #data= np.asarray(img, dtype="int32")
    data = np.array(img)
    gray = data[:,:,0]
    normalArray.append(gray)
    label =''
    print(np.array(normalArray).shape)
        

def abnormal_conv_Image(fileName):
    img = Image.open(abnormalData+fileName)
    img.load()
    #data= np.asarray(img, dtype="int32")
    data= np.array(img)
    gray = data[:,:,0]
    abnormalArray.append(gray)
    label = ''
    print(np.array(abnormalArray).shape)

def test_conv_Image(fileName):
    img = Image.open(testData+fileName)
    img.load()
    data = np.array(img)
    gray = data[:,:,0]
    testArray.append(gray) 

title: 1514765850_0_2018-01-01-00-17-30.jpg, id: 1n5K--PgmQ4W4FZHLAxFlgmilPhPl3Bad
title: 1514693400_0_2017-12-31-04-10-00.jpg, id: 137ONU9gf4ZgQzykZbJvJXWs2jhXLJd8B
title: 1514764176_0_2017-12-31-23-49-36.jpg, id: 1KLtb3ynmJ-7Zm84KYDIYzChUrYXNvc8o
title: 1514784000_0_2018-01-01-05-20-00.jpg, id: 1vv2wuTNu6khYQMeJ9iWtdoxwK9H8rF5O
title: 1514757600_0_2017-12-31-22-00-00.jpg, id: 1-nlMEPWiwGmZqkMCVMTS54UPOkXbeENk
title: 1514771400_0_2018-01-01-01-50-00.jpg, id: 1GpKQx53o87CXTnzNyStVO79KfBctk7F2
title: 1514763873_0_2017-12-31-23-44-33.jpg, id: 1pAtOjuJ0SLk8tTcwPQzXqOVXOrNKirVm
title: 1514752800_0_2017-12-31-20-40-00.jpg, id: 1BmcSgowjioW8n9GvrdOkxJO8K29iOWsr
title: 1514697300_0_2017-12-31-05-15-00.jpg, id: 1RKpySNJO0VaTxdYZRnHjmcbyGn9TPFJr
title: 1514834700_0_2018-01-01-19-25-00.jpg, id: 11MAtJct7YFXGGTpxdZOIYVXrEV5ZhfNH
title: 1514684100_0_2017-12-31-01-35-00.jpg, id: 1y71ovhNqEo8wtyqb-MlAuFx__bk3qNYX
title: 1514787600_0_2018-01-01-06-20-00.jpg, id: 1RsfmmBm8ma0b8a2l22XuVsOVJgBUWkwX
titl

Using TensorFlow backend.


(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 

(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 

(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 

(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 

In [2]:
#Run the image conversion and label the data
#for f in os.listdir(normalData):
#    if f.endswith(".jpg"):
#        normal_conv_Image(f)
#    else:
#        continue
        
#for f in os.listdir(abnormalData):
#    if f.endswith(".jpg"):
#        abnormal_conv_Image(f)
#    else:
#        continue

#for f in os.listdir(testData):
#    if f.endswith(".jpg"):
#        test_conv_Image(f)
#    else:
#        continue
        
#for x in np.nditer(np.array(normalArray)):
for x in np.ndindex(np.array(normalArray).shape[0]):
    label=0
    normal_Label.append(label)
normal_labels = np.array(normal_Label)
#for x in np.nditer(np.array(abnormalArray)):
for x in np.ndindex(np.array(abnormalArray).shape[0]):
    label =1
    abnormal_Label.append(label)
abnormal_labels=np.array(abnormal_Label)
print(np.array(normalArray).shape)
print(np.array(abnormalArray).shape)
labels_test=[(0, 0, 1, 0, 1, 1, 0, 0, 1, 0)]

print ('finished creating arrays and labels')

complete_data = np.concatenate((np.array(normalArray), np.array(abnormalArray)))
#complete_data = np.array(abnormalArray)
print (complete_data.shape[0])
print (complete_data.shape[1])
print (complete_data.shape[2])
complete_labels = np.concatenate((np.array(normal_labels), np.array(abnormal_labels)))
#complete_labels = abnormal_Label
print (complete_labels.shape)



(433, 540, 960, 3)
(23, 540, 960, 3)
finished creating arrays and labels
456
540
960
(456,)


In [0]:
#Create Estimator instance using model_load class
print ('creating Keras classifier')
#regressor = KerasRegressor(build_fn=model_load, nb_epoch=10, batch_size=64)
estimator = KerasClassifier(build_fn=conv_model_load, nb_epoch=100, batch_size=64)
print ('fitting Keras to labeled data')
#complete_data = complete_data.reshape(1,23,2)
#complete_data = complete_data.shape[1:]
print (np.array(complete_data))
#regressor.fit(np.array(complete_data), np.array(complete_labels), epochs=10)

#Fit estimator to data
estimator.fit(np.array(complete_data), np.array(complete_labels), epochs=100)

#Label for test data to determine test score 
#labels_test=[(1)]

#p = regressor.predict(np.array(testArray))
#score = regressor.score(np.array(testArray), np.array(labels_test))

#Prediction and Score
p = estimator.predict(np.array(testArray))
#score = estimator.score(np.array(testArray), np.array(labels_test))
print ('prediction for firework image: {}.'.format(p))
#print ('score for firework image: {}.'.format(score))


creating Keras classifier
fitting Keras to labeled data
[[[[ 92  67  45]
   [ 92  67  45]
   [ 92  67  45]
   ...
   [ 78  60  50]
   [ 78  60  50]
   [ 78  60  50]]

  [[ 92  67  45]
   [ 92  67  45]
   [ 92  67  45]
   ...
   [ 78  60  50]
   [ 78  60  50]
   [ 78  60  50]]

  [[ 92  67  45]
   [ 92  67  45]
   [ 92  67  45]
   ...
   [ 77  60  50]
   [ 77  60  50]
   [ 77  60  50]]

  ...

  [[190 175 152]
   [179 169 144]
   [173 165 142]
   ...
   [168 196 174]
   [179 202 176]
   [185 203 177]]

  [[189 177 153]
   [178 170 147]
   [175 168 149]
   ...
   [212 242 216]
   [240 255 231]
   [241 255 222]]

  [[183 171 147]
   [168 160 137]
   [161 154 135]
   ...
   [208 238 210]
   [240 255 224]
   [235 251 212]]]


 [[[123  99  73]
   [123  99  73]
   [122  98  72]
   ...
   [158 141 115]
   [158 141 115]
   [158 141 115]]

  [[123  99  73]
   [123  99  73]
   [122  98  72]
   ...
   [158 141 115]
   [158 141 115]
   [158 141 115]]

  [[123  99  73]
   [123  99  73]
   [122  98  